In [53]:
import json
import numpy as np
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
from keras.models import model_from_json
import pandas as pd

# we're still going to use a Tokenizer here, but we don't need to fit it
tokenizer = Tokenizer(num_words=3000)

labels = [0, 1]

# read in our saved dictionary
with open('dictionary.json', 'r') as dictionary_file:
    dictionary = json.load(dictionary_file)

# this utility makes sure that all the words in your input
# are registered in the dictionary
# before trying to turn them into a matrix.
def convert_text_to_index_array(text):
    words = kpt.text_to_word_sequence(text)
    wordIndices = []
    for word in words:
        if word in dictionary:
            wordIndices.append(dictionary[word])
    return wordIndices

# read in your saved model structure
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
# and create a model from that
model = model_from_json(loaded_model_json)
# and weight your nodes with your saved values
model.load_weights('model.h5')

In [54]:
tweets = pd.read_csv(r'nlp-getting-started\test_clean.csv', usecols = ['id','text'])

In [55]:
testArr = tweets['text'].apply(convert_text_to_index_array)
inputs = [tokenizer.sequences_to_matrix([x], mode='binary') for x in testArr]
pred = [model.predict(x) for x in inputs]
tweets['target'] = [labels[np.argmax(x)] for x in pred]

In [61]:
# make our first submission!
tweets[['id','target']].to_csv('nlp-getting-started/submission_1.csv', index=False)

In [58]:
tweets

,id,text,target
0,0,Just happened a terrible car crash,1
1,2,Heard about is different cities stay safe ever...,1
2,3,there is a forest fire at spot pond geese are ...,1
3,9,Apocalypse lighting,0
4,11,Typhoon Soudelor kills 28 in China and Taiwan,1
...,...,...,...
3258,10861,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,1
3259,10865,Storm in RI worse than last hurricane My citya...,1
3260,10868,Green Line derailment in Chicago,1
3261,10874,MEG issues Hazardous Weather Outlook HWO,1
